# Evalutation of the 200_cylindricalplates model

extract the emittances for all offsets and safe in a csv

In [1]:
import os
import numpy as np
import pandas as pd
import altair as alt
alt.enable_mime_rendering()
import scipy.constants

import PhaseSpaceEval.monitor_quantities as monq
from PhaseSpaceEval.import_particle_data import *
from PhaseSpaceEval.trajectory import Trajectory
from PhaseSpaceEval.particlemonitor import ParticleMonitor

## Import Raw Data and set filename *!!! ADJUST FILENAMES HERE !!!*

In [2]:
MODELNAME = "200_cylindricalplates"
RAW_PATH = "rawdata_" + MODELNAME + "/"
EMIT_FILENAME = "emit_" + MODELNAME + ".csv" # Name for the emittance output file
MON_FILENAME = "mon_" + MODELNAME + ".csv" # Name for the monitors output file

particle_source_names = import_source_names(RAW_PATH + MODELNAME + "-source_names.txt")
particle_constants = import_particle_constants(RAW_PATH + MODELNAME + "-constants.txt")
particle_trajectories = import_particle_trajectories(RAW_PATH + MODELNAME + "-trajectories.txt")

## Create convenient lists
### Raw Data

In [3]:
# Delete the single_centre source, not required
for key in particle_source_names.keys():
    if particle_source_names[key] == "single_centre":
        del(particle_source_names[key])
        break
        
# Generate simple list with all source IDs
sourceIDs = list(particle_source_names.keys())
#print(sourceIDs)

# Create lists with all particles belonging to a source and with the id of the central particles
particlesBySrc = dict() # dict for all particleIDs
centresBySrc = dict() # dict with the ids of the central particles
for sID in sourceIDs:
    pIDs = particle_constants["particleID"].loc[particle_constants["sourceID"] == sID].tolist()
    particlesBySrc.update({sID : pIDs})
    centresBySrc.update({sID : min(pIDs)}) # the smallest pID for each source is the centre
#print(particlesBySrc)
#print(centresBySrc)

### Trajectories

In [4]:
# Create Trajectories
trajsBySrc = dict() # Dict for all trajectories
ctrajsBySrc = dict() # Dict for central trajectories
lostParticles = list()
for sID in sourceIDs:
    # Compute central trajectory for each sID
    cID = centresBySrc[sID]
    ctr = Trajectory(particle_trajectories[cID],
                     particle_constants.loc[particle_constants["particleID"] == cID].squeeze())
    ctrajsBySrc.update({sID : ctr})

    # For each sID compute the trajectories of all pIDs
    # Note Particles that cannot be found in trajectory dataframe, these were lost
    pIDs = particlesBySrc[sID]
    trajs = list()
    for pID in pIDs:
        try:
            tr = Trajectory(particle_trajectories[pID],
                            particle_constants.loc[particle_constants["particleID"] == pID].squeeze())
        except KeyError:
            lostParticles.append(pID)
        trajs.append(tr)
    trajsBySrc.update({sID : trajs})
print(lostParticles)

[11838]


### Monitors

In [5]:
# Create Monitors
monBySrc = dict()
for sID in sourceIDs:
    ctr = ctrajsBySrc[sID]
    t0 = ctr.find_time("z", 501)
    mon = ParticleMonitor(time0=t0, trajectory=ctr)
    monBySrc.update({sID : mon})

## Record Monitor Interactions

In [6]:
# Record Monitor Interactions
for sID in sourceIDs:
    mon = monBySrc[sID]
    mon.reset_events()
    mon.reset_misses()
    for tr in trajsBySrc[sID]:
        mon.record_intersect(tr)

# Read Out misses and events
missesBySrc = dict()
eventsBySrc = dict()
for sID in sourceIDs:
    missesBySrc.update({sID : monBySrc[sID].get_misses()})
    eventsBySrc.update({sID : monBySrc[sID].get_events()})
#print(missesBySrc)

# Add lost particles to miss counter
for pID in lostParticles:
    for sID in sourceIDs:
        if pID in particlesBySrc[sID]:
            missesBySrc[sID] += 1
#print(missesBySrc)

## Export Monitor Interactions

In [7]:
export_events = pd.DataFrame()
for sID in sourceIDs:
    events = eventsBySrc[sID].copy()
    events["sourceID"] = sID
    events["sourceName"] = particle_source_names[sID]
    export_events = export_events.append(events, ignore_index=True)
export_events.to_csv(MON_FILENAME)

## Compute and save emittances

In [14]:
# Compute and save emittances
colnames = ["sourceID", "sourceName", "x_offset", "y_offset", "x_emittance", "y_emittance",
            "x_norm_emittance", "y_norm_emittance", "losses", "relbeta"]
emit_df = pd.DataFrame(columns=colnames)
emit_temp = pd.DataFrame([np.zeros(len(colnames))], columns=colnames)
for sID in sourceIDs:
    name = particle_source_names[sID]
    xoff = float(name.split('_')[1])
    yoff = float(name.split('_')[3])
    xemit = monq.emittance_u(eventsBySrc[sID])
    yemit = monq.emittance_v(eventsBySrc[sID])
    # Compute rel. beta for the screen 1e6 mm/ns ->m/s
    beta = monBySrc[sID].abs_vel / scipy.constants.speed_of_light * 1e6
    xemit_n = beta * (1 - beta**2)**(-0.5) * xemit # beta * gamme * emit
    yemit_n = beta * (1 - beta**2)**(-0.5) * yemit
    losses = missesBySrc[sID]
    
    emit_temp["sourceID"] = sID
    emit_temp["sourceName"] = name
    emit_temp["x_offset"] = xoff
    emit_temp["y_offset"] = yoff
    emit_temp["x_emittance"] = xemit
    emit_temp["y_emittance"] = yemit
    emit_temp["x_norm_emittance"] = xemit_n
    emit_temp["y_norm_emittance"] = yemit_n
    emit_temp["losses"] = losses
    emit_temp["relbeta"] = beta

    emit_df = emit_df.append(emit_temp, ignore_index=True)

emit_df.sort_values(["x_offset", "y_offset"], inplace=True)
emit_df.reset_index(inplace=True, drop=True)

# Load emittance data from startinplane to compute relative emittance growth
emit_start = pd.read_csv("emit_startingplane.csv", index_col=0)
emit_df["x_emittance_rel"] = emit_df["x_emittance"]/emit_start["x_emittance"]
emit_df["y_emittance_rel"] = emit_df["y_emittance"]/emit_start["y_emittance"]
# Save emittance table
emit_df.to_csv(EMIT_FILENAME)
# Show head for debugging purposes
emit_df

,sourceID,sourceName,x_offset,y_offset,x_emittance,y_emittance,x_norm_emittance,y_norm_emittance,losses,relbeta,x_emittance_rel,y_emittance_rel
0,16777222,x_-15_y_0,-15.0,0.0,1.659361,1.571046,0.003831,0.003627,0,0.002309,1.023580,0.999230
1,16777229,x_-15_y_5,-15.0,5.0,1.627251,1.597121,0.003757,0.003687,0,0.002309,1.021454,1.002159
2,16777236,x_-15_y_10,-15.0,10.0,1.653183,1.608146,0.003816,0.003712,0,0.002309,1.022306,1.007178
3,16777243,x_-15_y_15,-15.0,15.0,1.746339,1.670485,0.004031,0.003856,0,0.002309,1.044972,1.005179
4,16777220,x_-10_y_0,-10.0,0.0,1.591860,1.553274,0.003675,0.003586,0,0.002309,0.999832,1.000430
5,16777227,x_-10_y_5,-10.0,5.0,1.608500,1.582963,0.003713,0.003654,0,0.002309,0.997219,1.001844
6,16777234,x_-10_y_10,-10.0,10.0,1.490765,1.675788,0.003441,0.003869,0,0.002309,1.002758,1.006366
7,16777241,x_-10_y_15,-10.0,15.0,1.460472,1.639362,0.003372,0.003785,0,0.002309,1.007330,1.003108
8,16777218,x_-5_y_0,-5.0,0.0,1.681269,1.719681,0.003881,0.003970,0,0.002309,1.002930,1.001225
9,16777225,x_-5_y_5,-5.0,5.0,1.533805,1.637071,0.003541,0.003779,0,0.002309,1.006814,1.007002


## Plots

In [9]:
alt.Chart(emit_df).mark_line().encode(
    color='y_offset:N',
    x='x_offset',
    y='x_emittance',
)

<altair.VegaLite object>

In [10]:
alt.Chart(emit_df).mark_line().encode(
    color='y_offset:N',
    x='x_offset',
    y='y_emittance',
)

<altair.VegaLite object>

In [11]:
alt.Chart(emit_df).mark_line().encode(
    color='y_offset:N',
    x='x_offset',
    y='x_emittance_rel',
)

<altair.VegaLite object>

In [12]:
alt.Chart(emit_df).mark_line().encode(
    color='y_offset:N',
    x='x_offset',
    y='y_emittance_rel',
)

<altair.VegaLite object>

In [13]:
alt.Chart(emit_df).mark_line().encode(
    color='y_offset:N',
    x='x_offset',
    y='losses',
)

<altair.VegaLite object>